In [703]:
using HorizonSideRobots

In [704]:
a = Robot("untitled.sit", animate=true);

## a ##

In [705]:
function HorizonSideRobots.move!(r, side, n)
    for _ in range(1, n)
        move!(r, side)
    end
end

In [706]:
function inverse(side)
    if side == Ost
        return West
    end
    if side == West
        return Ost
    end
    if side == Sud
        return Nord
    end
    if side == Nord
        return Sud
    end
end

inverse (generic function with 1 method)

In [707]:
function n_steps_along!(stop_cond, r, side)
    n = 0
    while !stop_cond()
        move!(r, side)
        n += 1
    end
    return n
end

n_steps_along! (generic function with 1 method)

In [708]:
function along!(stop_cond, r, side)
    while !stop_cond()
        move!(r, side)
    end
end

along! (generic function with 1 method)

In [709]:
function to_start(r)
    n_west = []
    n_sud = []
    while !(isborder(r, Sud) && isborder(r, West))
        append!(n_west, n_steps_along!(() -> isborder(r, West), r, West))
        append!(n_sud, n_steps_along!(() -> isborder(r, Sud), r, Sud))
    end
    return n_west, n_sud
end

to_start (generic function with 1 method)

In [710]:
function fill_side(r, side)
    while !isborder(r, side)
        putmarker!(r)
        move!(r, side)
    end
end

fill_side (generic function with 1 method)

In [711]:
function to_f_pos(r, arr_w, arr_s)
    for i in range(-length(arr_w), -1)
        move!(r, Nord, arr_s[abs(i)])
        move!(r, Ost, arr_w[abs(i)])
    end
end

to_f_pos (generic function with 1 method)

In [712]:
function main_a(r)
    n_west, n_sud = to_start(r)
    fill_side(r, Nord)
    fill_side(r, Ost)
    fill_side(r, Sud)
    fill_side(r, West)
    to_f_pos(r, n_west, n_sud)
end

main_a (generic function with 1 method)

## b ##

In [1]:
function count_holes(r, side)
    if side == West || side == Ost
        side2 = Nord
        side3 = Sud
    else
        side2 = West
        side3 = Ost
    end
    flag_1 = 0
    flag_2 = 0
    steps = 0
    while isborder(r, side) && !isborder(r, side2)
        move!(r, side2)
        steps += 1
        if !isborder(r, side)
            flag_1 = 1
        end
    end
    if flag_1 == 0
        move!(r, side3, steps)
        steps = 0
        while isborder(r, side) && !isborder(r, side3)
            move!(r, side3)
            steps += 1
            if !isborder(r, side)
                flag_2 = 1
            end
        end
    end
    if flag_1 == 0 && flag_2 == 0
        move!(r, side2, steps)
        return true
    else 
        n_st_along = n_steps_along!(() -> isborder(r, side), r, side)
        if flag_1 == 1
            move!(r, side3, steps)
            return false, [side2, steps], n_st_along
        elseif flag_2 == 1
            move!(r, side2, steps)
            return false, [side3, steps], n_st_along
        end
    end
end

count_holes (generic function with 1 method)

In [714]:
function mark_at_side(r, side)
    res = 0
    way_back = []
    steps_a = []
    push!(steps_a, n_steps_along!(() -> isborder(r, side), r, side))
    while res[1] != true
        res = count_holes(r, side)
        if length(res) != 1
            push!(way_back, res[2])
            push!(steps_a, res[3])
        end
    end
    putmarker!(r)
    if length(way_back) > 0
        for i in range(1, length(steps_a)-1)
            move!(r, way_back[i][1], way_back[i][2])
            move!(r, inverse(side), steps_a[abs(i)+1])
            move!(r, inverse(way_back[i][1]), way_back[i][2])
        end
        if length(steps_a) > 1
            move!(r, inverse(side), steps_a[1])
        end
    else
        move!(r, inverse(side), steps_a[1])
    end
end

mark_at_side (generic function with 1 method)

In [715]:
function main_b(r)
    mark_at_side(r, West)
    mark_at_side(r, Nord)
    mark_at_side(r, Sud)
    mark_at_side(r, Ost)
end

main_b (generic function with 1 method)